# Client Side

In [2]:
! pip install rssi
! pip install psutil

The utilization data is captured using the `psutil` library 

In [3]:
import psutil as ps
import threading as th

## Node info

the Node utilization funtion of $Node_i$ is $C_i^N(t) = f_i^N(X_i^N,t) = \bigg[ \alpha_c \big( fc_i(t)*nc_i(t)*uc_i(t) \big) + \alpha_m \big( fm_i(t)*vm_i(t)*um_i(t) \big) \bigg]$, where the parameters are,

| Resource | Node parameters | meaning |
|---|---|---|
|CPU| $fc_i(t)$ | max core frequency  |
|| $nc_i(t)$ | number of logical core |
|| $uc_i(t)$ | mean CPU utilization at time $t$ |
|Memory| $fm_i(t)$ | max DRAM frequency |
|| $vm_i(t)$ | RAM volume $t$ |
|| $um_i(t)$ | mean RAM utilization at time $t$ |

### CPU info

In [43]:
import pprint
pp =pprint.PrettyPrinter(indent=4)

In [4]:
# number of logical cpu cores
ps.cpu_count(logical=True)

8

In [5]:
# Max frequency 
ps.cpu_freq(percpu=True)

[scpufreq(current=4000.0, min=0.0, max=4001.0)]

In [6]:
ps.cpu_freq(percpu=True)[0][2]  # max freq

4001.0

In [7]:
# average utilization
ps.cpu_percent(interval=1, percpu=True)

[14.9, 9.2, 14.1, 6.2, 17.2, 3.1, 10.9, 15.6]

In [8]:
round(sum(ps.cpu_percent(interval=1, percpu=True))/ps.cpu_count(),4)

11.825

In [9]:
def get_cpu_freq():
    temp = ps.cpu_freq(percpu=True)[0]
    return {'current' :temp[0], 'min': temp[1], 'max': temp[2] }

In [10]:
def get_cpu_util():
    temp = ps.cpu_percent(interval=1, percpu=True)
    return {'per_core_util' : temp, 'mean_util' : round(sum(temp)/len(temp) , 4)}

In [11]:
%%time
node_util = {
    'cpu' : {
        'util': get_cpu_util(),
        'freq': get_cpu_freq(),
        'core_count' : ps.cpu_count(),
    }
}

Wall time: 1 s


In [44]:
%%time
pp.pprint(node_util)

{   'cpu': {   'core_count': 8,
               'freq': {'current': 1900.0, 'max': 4001.0, 'min': 0.0},
               'util': {   'mean_util': 10.7,
                           'per_core_util': [   20.9,
                                                6.2,
                                                15.4,
                                                7.7,
                                                16.9,
                                                4.6,
                                                6.2,
                                                7.7]}},
    'memory': {'total_mb': 32639.1133, 'util': 37.4}}
Wall time: 996 µs


### Memory Info 

In [46]:
ps.virtual_memory()

svmem(total=34224590848, available=21331185664, percent=37.7, used=12893405184, free=21331185664)

In [47]:
def get_mem_info():
    temp = ps.virtual_memory()
    return {'total_mb': round(temp[0]/(1024*1024), 4), 'util':temp[2]}

In [48]:
%%time
node_util = {
    'cpu' : {
        'util': get_cpu_util(),
        'freq': get_cpu_freq(),
        'core_count' : ps.cpu_count(),
    },
    'memory': get_mem_info()
}

Wall time: 1 s


In [50]:
pp.pprint(node_util)

{   'cpu': {   'core_count': 8,
               'freq': {'current': 800.0, 'max': 4001.0, 'min': 0.0},
               'util': {   'mean_util': 9.9375,
                           'per_core_util': [   10.8,
                                                4.7,
                                                17.2,
                                                6.2,
                                                14.1,
                                                3.1,
                                                6.2,
                                                17.2]}},
    'memory': {'total_mb': 32639.1133, 'util': 37.7}}


## Network 

In [17]:
iface = 'Wi-Fi'

In [18]:
ps.net_if_stats()

{'Ethernet': snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=1000, mtu=1468),
 'VMware Network Adapter VMnet1': snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=100, mtu=1500),
 'VMware Network Adapter VMnet8': snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=100, mtu=1500),
 'VirtualBox Host-Only Network': snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=1000, mtu=1500),
 'Loopback Pseudo-Interface 1': snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=1073, mtu=1500),
 'Wi-Fi': snicstats(isup=False, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=65, mtu=1500),
 'Local Area Connection* 1': snicstats(isup=False, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=0, mtu=1500),
 'Local Area Connection* 10': snicstats(isup=False, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=0, mtu=1500)}

In [19]:
# getting the interface names only

ps.net_if_stats().keys()

dict_keys(['Ethernet', 'VMware Network Adapter VMnet1', 'VMware Network Adapter VMnet8', 'VirtualBox Host-Only Network', 'Loopback Pseudo-Interface 1', 'Wi-Fi', 'Local Area Connection* 1', 'Local Area Connection* 10'])

In [24]:
ps.net_if_stats()[iface]

snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=1000, mtu=1468)

In [26]:
# global ds to hold nic info
nic_result={}

In [37]:
# getting stats for all interfaces 

for iface in ps.net_if_stats().keys():
    stat  = ps.net_if_stats()[iface]
    nic_result[iface] = {
        'state' : stat[0],
        'duplex' : stat[1],
         'speed' : stat[2],
         'mtu': stat[3]
    }

In [51]:
pp.pprint(nic_result)

{   'Ethernet': {   'addr': {   'ip4': '10.33.30.201',
                                'mac': '38-D5-47-0F-78-22',
                                'mask': '255.255.254.0'},
                    'stats': {   'duplex': <NicDuplex.NIC_DUPLEX_FULL: 2>,
                                 'mtu': 1468,
                                 'speed': 1000,
                                 'state': True}},
    'Local Area Connection* 1': {   'addr': {   'ip4': '169.254.216.188',
                                                'mac': 'EA-4E-06-57-08-67',
                                                'mask': '255.255.0.0'},
                                    'stats': {   'duplex': <NicDuplex.NIC_DUPLEX_FULL: 2>,
                                                 'mtu': 1500,
                                                 'speed': 0,
                                                 'state': False}},
    'Local Area Connection* 10': {   'addr': {   'ip4': '169.254.154.154',
                              

In [39]:
# appending iface info
for iface in ps.net_if_stats().keys():
    stat  = ps.net_if_stats()[iface]
    addr =  ps.net_if_addrs()[iface]

    nic_result[iface] = {
        'stats': {
            'state' : stat[0],
            'duplex' : stat[1],
            'speed' : stat[2],
            'mtu': stat[3]
        },
        'addr': {
            'mac' : addr[0][1],
            'ip4' : addr[1][1],
            'mask': addr[1][2]
        }
    }

In [42]:
pp.pprint(nic_result)

{   'Ethernet': {   'addr': {   'ip4': '10.33.30.201',
                                'mac': '38-D5-47-0F-78-22',
                                'mask': '255.255.254.0'},
                    'stats': {   'duplex': <NicDuplex.NIC_DUPLEX_FULL: 2>,
                                 'mtu': 1468,
                                 'speed': 1000,
                                 'state': True}},
    'Local Area Connection* 1': {   'addr': {   'ip4': '169.254.216.188',
                                                'mac': 'EA-4E-06-57-08-67',
                                                'mask': '255.255.0.0'},
                                    'stats': {   'duplex': <NicDuplex.NIC_DUPLEX_FULL: 2>,
                                                 'mtu': 1500,
                                                 'speed': 0,
                                                 'state': False}},
    'Local Area Connection* 10': {   'addr': {   'ip4': '169.254.154.154',
                              

In [41]:
ps.net_if_addrs()[iface]

[snicaddr(family=<AddressFamily.AF_LINK: -1>, address='E8-4E-06-57-08-67', netmask=None, broadcast=None, ptp=None),
 snicaddr(family=<AddressFamily.AF_INET: 2>, address='169.254.154.154', netmask='255.255.0.0', broadcast=None, ptp=None),
 snicaddr(family=<AddressFamily.AF_INET6: 23>, address='fe80::6cfa:e601:dacd:9a9a', netmask=None, broadcast=None, ptp=None)]

In [59]:
ps.net_io_counters(pernic=True,nowrap=True)

{'Ethernet': snetio(bytes_sent=63069849, bytes_recv=7267290695, packets_sent=885743, packets_recv=196966, errin=0, errout=0, dropin=0, dropout=0),
 'VirtualBox Host-Only Network': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'Local Area Connection* 1': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'Local Area Connection* 10': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'VMware Network Adapter VMnet1': snetio(bytes_sent=948, bytes_recv=0, packets_sent=948, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'VMware Network Adapter VMnet8': snetio(bytes_sent=811, bytes_recv=10, packets_sent=811, packets_recv=10, errin=0, errout=0, dropin=0, dropout=0),
 'Wi-Fi': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'Loopback Pseudo-Interfa

In [72]:
# appending counter info

for iface in ps.net_if_stats().keys():
    stat  = ps.net_if_stats()[iface]
    addr =  ps.net_if_addrs()[iface]
    count = ps.net_io_counters(pernic=True)[iface]

    nic_result[iface] = {
        'stats': {
            'state' : stat[0],
            'duplex' : stat[1],
            'speed' : stat[2],
            'mtu': stat[3]
        },
        'addr': {
            'mac' : addr[0][1],
            'ip4' : addr[1][1],
            'mask': addr[1][2]
        },
        'counter' : {
            'byte' : { 'tx' : count[0], 'rx' : count[1]},
            'packet' : { 'tx' : count[2], 'rx' : count[3]},
            'error' : {'tx' : count[5], 'rx' : count[4]},
            'drop' : { 'tx' : count[7], 'rx' : count[6] }
        }
    }

In [73]:
pp.pprint(nic_result)

{   'Ethernet': {   'addr': {   'ip4': '10.33.30.201',
                                'mac': '38-D5-47-0F-78-22',
                                'mask': '255.255.254.0'},
                    'counter': {   'byte': {'rx': 7267866961, 'tx': 63359507},
                                   'drop': {'rx': 0, 'tx': 0},
                                   'error': {'rx': 0, 'tx': 0},
                                   'packet': {'rx': 199256, 'tx': 886774}},
                    'stats': {   'duplex': <NicDuplex.NIC_DUPLEX_FULL: 2>,
                                 'mtu': 1468,
                                 'speed': 1000,
                                 'state': True}},
    'Local Area Connection* 1': {   'addr': {   'ip4': '169.254.216.188',
                                                'mac': 'EA-4E-06-57-08-67',
                                                'mask': '255.255.0.0'},
                                    'counter': {   'byte': {'rx': 0, 'tx': 0},
                         

In [75]:
def get_iface_info():
    nic_result={}

    for iface in ps.net_if_stats().keys():
        stat  = ps.net_if_stats()[iface]
        addr =  ps.net_if_addrs()[iface]
        count = ps.net_io_counters(pernic=True)[iface]

        nic_result[iface] = {
            'stats': {
                'state' : stat[0],
                'duplex' : stat[1],
                'speed' : stat[2],
                'mtu': stat[3]
            },
            'addr': {
                'mac' : addr[0][1],
                'ip4' : addr[1][1],
                'mask': addr[1][2]
            },
            'counter' : {
                'byte' : { 'tx' : count[0], 'rx' : count[1]},
                'packet' : { 'tx' : count[2], 'rx' : count[3]},
                'error' : {'tx' : count[5], 'rx' : count[4]},
                'drop' : { 'tx' : count[7], 'rx' : count[6] }
            }
        }

    return nic_result

In [83]:
'''
def get_iface_info(iface):
    addr = ps.net_if_addrs()[iface]
    stats = ps.net_if_stats()[iface]
    count = ps.net_io_counters(pernic=True,nowrap=True)[iface]
    return {
        'addr' : {
            'ip': addr[1][1],
            'mask': addr[1][2],
            'mac': addr[0][1]
        },
        'stat' : {
            'staus': stats[0],
            'duplex': stats[1],
            'speed':stats[2],
            'mtu':stats[3],
        },
        'counter' : {
            'byte' : { 'tx' : count[0], 'rx' : count[1]},
            'packet' : { 'tx' : count[2], 'rx' : count[3]},
            'error' : {'tx' : count[5], 'rx' : count[4]},
            'drop' : { 'tx' : count[7], 'rx' : count[6] }
        }
    }
''' 

In [80]:
%%time
def get_util():
    node_util = {
        'cpu' : {
            'util': get_cpu_util(),
            'freq': get_cpu_freq(),
            'core_count' : ps.cpu_count(),
        },
        'memory': get_mem_info(),
        'network': get_iface_info()
    }
    return node_util

Wall time: 0 ns


In [81]:
%%time
pp.pprint(get_util())

{   'cpu': {   'core_count': 8,
               'freq': {'current': 800.0, 'max': 4001.0, 'min': 0.0},
               'util': {   'mean_util': 11.7375,
                           'per_core_util': [   22.1,
                                                3.1,
                                                17.2,
                                                6.2,
                                                12.5,
                                                12.5,
                                                9.4,
                                                10.9]}},
    'memory': {'total_mb': 32639.1133, 'util': 37.7},
    'network': {   'Ethernet': {   'addr': {   'ip4': '10.33.30.201',
                                               'mac': '38-D5-47-0F-78-22',
                                               'mask': '255.255.254.0'},
                                   'counter': {   'byte': {   'rx': 7268223653,
                                                              'tx

In [111]:
import json
b = str.encode(json.dumps(get_util(iface='Wi-Fi')))
s = json.loads(bytes.decode(b))

print(b,'\n\n',s)

b'{"cpu": {"util": {"per_core_util": [10.8, 6.2, 21.5, 1.5, 12.3, 4.6, 6.2, 9.2], "mean_util": 9.0375}, "freq": {"current": 800.0, "min": 0.0, "max": 4001.0}, "core_count": 8}, "memory": {"total_mb": 32639.1133, "util": 36.5}, "network": {"addr": {"ip": "136.148.146.86", "mask": "255.255.252.0", "mac": "E8-4E-06-57-08-67"}, "stat": {"staus": true, "duplex": 2, "speed": 52, "mtu": 1500}, "counter": {"byte": {"tx": 1327943689, "rx": 2972065167}, "packet": {"tx": 996559, "rx": 2993648}, "error": {"tx": 0, "rx": 0}, "drop": {"tx": 0, "rx": 0}}}}' 

 {'cpu': {'util': {'per_core_util': [10.8, 6.2, 21.5, 1.5, 12.3, 4.6, 6.2, 9.2], 'mean_util': 9.0375}, 'freq': {'current': 800.0, 'min': 0.0, 'max': 4001.0}, 'core_count': 8}, 'memory': {'total_mb': 32639.1133, 'util': 36.5}, 'network': {'addr': {'ip': '136.148.146.86', 'mask': '255.255.252.0', 'mac': 'E8-4E-06-57-08-67'}, 'stat': {'staus': True, 'duplex': 2, 'speed': 52, 'mtu': 1500}, 'counter': {'byte': {'tx': 1327943689, 'rx': 2972065167}, 'p

In [91]:
import time 
import random
import threading
import queue

def f1(a,b):
    time.sleep(random.randint(0,4))
    print(a+b)
    return a+b
def f2(a,b):
    time.sleep(random.randint(0,4))
    print(a*b)
    return a*b

T = []
R = {}
Q = queue.Queue()
T.append(threading.Thread(target=f1,args=(5,6,Q)))
T.append(threading.Thread(target=f2,args=(5,6,Q)))

for t in T:
    t.start()
    
for t in T:
    t.join()
    
'''    
R['add'] = T[0].join()
R['mul'] = T[1].join()
'''
print(R)


3011

None
None
{}


In [89]:
R

{'add': None, 'mul': None}